libraries

In [1]:
import cv2 as cv
import os
import numpy as np
import math
import random
import shutil

pre trained models

In [2]:
models = cv.CascadeClassifier('./pre-trained-models/haarcascade_frontalface_default.xml')

dataset

In [3]:
dataset_path = './Dataset'

train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')

os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

train_ratio = 0.75

for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    
    if not os.path.isdir(folder_path):
        continue
    
    images = os.listdir(folder_path)
    
    # number of train images
    n = int(len(images) * train_ratio)
    
    # random shuffle
    random.shuffle(images)
    
    # SPLIT DATA
    train_img = images[:n]
    test_img = images[n:]
    
    # create separate folder in train folder
    train_folder = os.path.join(train_path, folder_name)
    os.makedirs(train_folder, exist_ok=True)
    
    # copy train images to train folder
    for img in train_img:
        source_path = os.path.join(folder_path, img)
        dest_path = os.path.join(train_folder, img)
        shutil.copy(source_path, dest_path)
        
    # copy to test images to test folder
    for img in test_img:
        source_path = os.path.join(folder_path, img)
        # rename test images just incase duplicate name
        dest_path = os.path.join(test_path, f'{folder_name}-{img}')
        shutil.copy(source_path, dest_path)

PermissionError: [Errno 13] Permission denied: './Dataset\\train\\karim_benzema'

In [4]:
# delete additional folders that will be made
shutil.rmtree('./Dataset/train/train')
shutil.rmtree('./Dataset/train/test')

face detection

In [5]:
train_path = './Dataset/train/'
train_folder = os.listdir(train_path)

In [6]:
faces = []
labels = []
    
for idx, folders in enumerate(train_folder):
    filenames = os.path.join(train_path, folders)
        
    for filename in os.listdir(filenames):
        path = os.path.join(filenames, filename)
        img_gray = cv.imread(path, cv.IMREAD_GRAYSCALE)

        face = models.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5)
            
            
            
        if(len(face) < 1):
            continue
        else:
            for rect in face:
                x, y, w, h = rect
                face_image = img_gray[y:y+h, x:x+w]
                faces.append(face_image)
                labels.append(idx)
            

In [8]:
# FACE RECOGNITION
recognizer = cv.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(labels))
    
    # TESTING
test_path = './Dataset/test/'
    
accuracy_list = []
    
for filename in os.listdir(test_path):
    img_path = f"{test_path}/{filename}"
    test_img = cv.imread(img_path)
    img_gray = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
        
    face = models.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=3)
        
    if(len(face) < 1):
        continue
    else:
        for face_rect in face:
            x, y, w, h = face_rect
            face_image = img_gray[y:y+h, x:x+w]

            res, conf = recognizer.predict(face_image)
                
            accuracy_list.append(conf)
        
    
average_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f"Average Accuracy = {float(average_accuracy)}")

Average Accuracy = 59.14142724218241
